In [ ]:
#подключаем нужные нам библиотеки
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
#получаем необходимые данные
data = pd.read_csv("https://video.ittensive.com/python-advanced/rts-index.csv")
#преобразуем строковые значения в серии данных "Date", к типу данных "дата" 
#указываем dayfirst=True, чтобы все даты корректно преобразовались из русского формата, в английский
data["Date"] = pd.to_datetime(data["Date"], dayfirst=True)
#выполняем сортировку данных, от минимального к максимального
dates = pd.date_range(min(data["Date"]), max(data["Date"]))
#в качестве индекса используем серию данных "Date"
data = data.set_index("Date")
#переиндексируем все данные, заполним пустые даты предыдущими значениями
#это понадобиться нам для сравнения данных по годам, чтобы в каждый день года
#у нас было хотя бы одно значение
data = data.reindex(dates).ffill()
#добавим ещё одну серию данных, "день года" для подписи по осям x
data["Day"] = pd.to_datetime(data.index).dayofyear
#назначаем название индекса, которое потерялось при переиндексации данных
data.index.name = "Date"
#сортируем по индексу, чтобы развернуть данные в правильном хронологическом порядке
data = data.sort_index()
#для данных за 2019 год создадим отдельный набор, чтобы в последствии искать искомую
#дату превышения, как пересечение его с набором за 2017 год
data_2019 = data["2019"].reset_index().set_index("Day")
#аналогичным образом и создадим набор данных и для 2017 года,
#для него сразу возьмём экспоненциальное среднее, со сдвигом 20 от значение максимума
data_2017 = data["2017"].reset_index().set_index("Day")["Max"].ewm(span=20).mean()
#создаём наш холст, размером 12 на 8
fig = plt.figure(figsize=(12,8))
#добавляем на него 1 столбец, 1 строка, 1 подобласть
area = fig.add_subplot(1, 1, 1)
#наносим данные на область area, с названием "2019" цвет красный, толщина 3 
data_2019["Close"].plot(ax=area, color="red", label="2019", lw=3)
#наносим обычные данные за 2017 год в виде оранжевой линии.
data_2017.plot(ax=area, color="orange", label="Exp.2017", lw=3)
#переопределяем индекс, и назначаем в качестве индекса серию данных "Close"
data["2017"].reset_index().set_index("Day")["Close"].plot.area(ax=area, color=".5", label="2017")
#а также за 2018 год, виде ещё одной линии
data["2018"].reset_index().set_index("Day")["Close"].plot(ax=area, color="blue", label="2018", lw=3)
#добавляем легенду
plt.legend()
plt.show()
#находим дату, когда показатели индекса РТС, превзошли показатели за 2017 год
#фильтруем данные за 2019 год, когда "Close" (закрытие дня), было больше максимума 
# аналогичного дня за 2017 год  
data_fall = data_2019[data_2019["Close"] < data_2017[0:len(data_2019)]]
#выставляем индекс по дате
data_fall.set_index("Date", inplace=True)
#отсортируем по индексу
data_fall = data_fall.sort_index(ascending=False)
#выведем первое значение, то есть последнюю дату, когда дневной максимум в 2017 году
#был больше закрытия, в соответствующим дне 2019 года
print (data_fall.head(1).index)